In [22]:
import ipywidgets as widgets
from IPython.display import display, clear_output

output = widgets.Output()

t = widgets.Text(
    description='Mot clé:',
    value='data',
    disabled=False   
)
n = widgets.IntText( 
    description="Nb articles",
    value=10,
    min=2
)
button = widgets.Button(
    description='Valider',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Valider',
)



# Fonction de gestionnaire d'événements pour le changement de valeur des champs
def update_form(change):
    if len(t.value.strip()) > 0 and n.value > 0:
        # Afficher le message lorsque tous les champs sont remplis
        clear_output(wait=True)
        
    else:
        # Afficher le formulaire tant que tous les champs ne sont pas remplis
        clear_output(wait=True)
        display(form)

# Liaison de la fonction de gestionnaire d'événements aux changements de valeur des champs
t.observe(update_form, names='value')
n.observe(update_form, names='value')

# Création du formulaire
form = widgets.VBox([t, n])

display(form)


In [23]:

# =============== PARTIE 1 =============
# =============== 1.1 : REDDIT ===============
# Libraries
import praw
import urllib, urllib.request
import xmltodict
import datetime
from Classes import ArxivDocument, RedditDocument

limit = n.value
keyword = t.value
# Fonction affichage hiérarchie dict
def showDictStruct(d):
    def recursivePrint(d, i):
        for k in d:
            if isinstance(d[k], dict):
                print("-"*i, k)
                recursivePrint(d[k], i+2)
            else:
                print("-"*i, k, ":", d[k])
    recursivePrint(d, 1)

def rechercher_documents():
    
    # Identification
    reddit = praw.Reddit(client_id='QyizTPZdvFMfbmCtobzj1g', client_secret='TkvCDzi73SAaAge2LDOgEFHb6sF0NA', user_agent='test')

    hot_posts = reddit.subreddit(keyword).hot(limit=limit)#.top("all", limit=limit)#

    # Récupération du texte
    docs = []
    docs_bruts = []
    afficher_cles = False
    for i, post in enumerate(hot_posts):
        if i%10==0: print("Reddit:", i, "/", limit)
        if afficher_cles:  # Pour connaître les différentes variables et leur contenu
            for k, v in post.__dict__.items():
                pass
                print(k, ":", v)

        if post.selftext != "":  # Osef des posts sans texte
            pass
            #print(post.selftext)
        docs.append(post.selftext.replace("\n", " "))
        docs_bruts.append(("Reddit", post))

    #print(docs)

    # =============== 1.2 : ArXiv ===============

    # Paramètres
    query_terms = [keyword]

    # Requête
    url = f'http://export.arxiv.org/api/query?search_query=all:{"+".join(query_terms)}&start=0&max_results={limit}'
    data = urllib.request.urlopen(url)

    # Format dict (OrderedDict)
    data = xmltodict.parse(data.read().decode('utf-8'))

    #showDictStruct(data)

    # Ajout résumés à la liste
    for i, entry in enumerate(data["feed"]["entry"]):
        if i%10==0: print("ArXiv:", i, "/", limit)
        docs.append(entry["summary"].replace("\n", ""))
        docs_bruts.append(("ArXiv", entry))
        #showDictStruct(entry)

    # =============== 1.3 : Exploitation ===============
    print(f"# docs avec doublons : {len(docs)}")
    docs = list(set(docs))
    print(f"# docs sans doublons : {len(docs)}")

    for i, doc in enumerate(docs):
        print(f"Document {i}\t# caractères : {len(doc)}\t# mots : {len(doc.split(' '))}\t# phrases : {len(doc.split('.'))}")
        if len(doc)<100:
            docs.remove(doc)

    longueChaineDeCaracteres = " ".join(docs)

    # =============== PARTIE 2 =============

    # =============== 2.3 : MANIPS ===============



    collection = []

    for nature, doc in docs_bruts:
        if nature == "ArXiv":  # Les fichiers de ArXiv ou de Reddit sont pas formatés de la même manière à ce stade.
            #showDictStruct(doc)

            titre = doc["title"].replace('\n', '')  # On enlève les retours à la ligne
            try:
                authors = ", ".join([a["name"] for a in doc["author"]])  # On fait une liste d'auteurs, séparés par une virgule
            except:
                authors = doc["author"]["name"]  # Si l'auteur est seul, pas besoin de liste
            summary = doc["summary"].replace("\n", "")  # On enlève les retours à la ligne
            date = datetime.datetime.strptime(doc["published"], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y/%m/%d")  # Formatage de la date en année/mois/jour avec librairie datetime

            # doc_classe = Document(titre, authors, date, doc["id"], summary)  # Création du Document
            # Adding ArxivDocument
            arxiv_doc = ArxivDocument(title=titre, auteur=authors, date=date, url=doc["id"], texte=summary)

            collection.append(arxiv_doc)  # Ajout du Document à la liste.

        elif nature == "Reddit":
            #print("".join([f"{k}: {v}\n" for k, v in doc.__dict__.items()]))
            titre = doc.title.replace("\n", '')
            auteur = str(doc.author)
            date = datetime.datetime.fromtimestamp(doc.created).strftime("%Y/%m/%d")
            url = "https://www.reddit.com/"+doc.permalink
            texte = doc.selftext.replace("\n", "")
            num_comments = int(doc.num_comments)
            # Adding RedditDocument
            reddit_doc = RedditDocument(title=titre, auteur=auteur, date=date, url=url, texte=texte, num_comments=num_comments)
            # doc_classe = Document(titre, auteur, date, url, texte)
            
            collection.append(reddit_doc)

    # Création de l'index de documents
    id2doc = {}
    for i, doc in enumerate(collection):
        id2doc[i] = doc.titre

    # =============== 2.4, 2.5 : CLASSE AUTEURS ===============
    from Classes import Author

    # =============== 2.6 : DICT AUTEURS ===============
    authors = {}
    aut2id = {}
    num_auteurs_vus = 0

    # Création de la liste+index des Auteurs
    for doc in collection:
        if doc.auteur not in aut2id:
            num_auteurs_vus += 1
            authors[num_auteurs_vus] = Author(doc.auteur)
            aut2id[doc.auteur] = num_auteurs_vus

        authors[aut2id[doc.auteur]].add(doc.texte)


    # =============== 2.7, 2.8 : CORPUS ===============
    from Corpus import Corpus
    corpus = Corpus("Mon corpus")

    # Construction du corpus à partir des documents
    for doc in collection:
        corpus.add(doc,)
    #corpus.show(tri="abc")
    #print(repr(corpus))

    # =============== 4.1 : TEST SINGLETON ===============
    # corpus_test = Corpus("Mon corpus test")
    # print("Instance unique de corpus ?", corpus is corpus_test)

    # =============== 2.9 : SAUVEGARDE ===============
    import pickle

    # Ouverture d'un fichier, puis écriture avec pickle
    with open("corpus.pkl", "wb") as f:
        pickle.dump(corpus, f)

    # Supression de la variable "corpus"
    del corpus

    # Ouverture du fichier, puis lecture avec pickle
    with open("corpus.pkl", "rb") as f:
        corpus = pickle.load(f)

    # La variable est réapparue
    print(corpus)

if len(keyword) > 0:
    button.on_click(rechercher_documents()) 
else:
    print("Le mot clé est requis svp.")




Reddit: 0 / 10
ArXiv: 0 / 10
# docs avec doublons : 20
# docs sans doublons : 20
Document 0	# caractères : 0	# mots : 1	# phrases : 1
Document 1	# caractères : 635	# mots : 102	# phrases : 5
Document 2	# caractères : 803	# mots : 128	# phrases : 7
Document 3	# caractères : 315	# mots : 52	# phrases : 3
Document 4	# caractères : 166	# mots : 29	# phrases : 3
Document 5	# caractères : 1768	# mots : 261	# phrases : 18
Document 6	# caractères : 836	# mots : 119	# phrases : 8
Document 7	# caractères : 939	# mots : 135	# phrases : 9
Document 8	# caractères : 912	# mots : 115	# phrases : 7
Document 9	# caractères : 1139	# mots : 161	# phrases : 8
Document 10	# caractères : 270	# mots : 49	# phrases : 3
Document 11	# caractères : 1135	# mots : 243	# phrases : 9
Document 12	# caractères : 1238	# mots : 182	# phrases : 6
Document 13	# caractères : 294	# mots : 46	# phrases : 3
Document 14	# caractères : 988	# mots : 134	# phrases : 7
Document 15	# caractères : 399	# mots : 72	# phrases : 4
Docum